Zadaniem na zajęcia 2 jest implementacja działającego na zbiorze heart disease 
modelu klasyfikacji, przy czym: <br>
 
• Zbieżność modelu można zdefiniować przez wystarczająco małą zmianę 
funkcji kosztu w danej iteracji i pewną maksymalną liczbę iteracji <br>
• Model  może  uczyć  się  wyliczając  sumaryczną  pochodną  z  funkcji  kosztu  po 
całym zbiorze, po jednym przykładzie lub po paczce przykładów w iteracji. Dla 
sieci  neuronowej  w  następnym  zadaniu  będzie  już  wymagany  tryb 
paczkowania,  więc  warto  przećwiczyć  operacje  na  całych  macierzach  a  nie 
tylko pojedynczych wektorach danych <br>
• Uczenie się modelu powinno być weryfikowalne metryką (np. accuracy, fscore, 
precision – można korzystać z bibliotek) <br>
• Weryfikacja powinna uwzględnić podział na dane uczące i testowe

In [44]:
from ucimlrepo import fetch_ucirepo
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import missingno as msno
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, f1_score

heart_disease = fetch_ucirepo(id=45)

# Porzucenie linii z pustymi etykietami oraz odpowiadajacych im wartosci
x = heart_disease.data.features.dropna()
y = heart_disease.data.targets.loc[x.index]

# Utworzenie zbioru wartości binarnych i wartości w formie dummy
y_binary = y.copy()
y_binary['num'] = y_binary['num'].apply(lambda x: 1 if x != 0 else 0)
y_dummy = pd.get_dummies(y, columns=['num'])

# Utworznnie zbioru dummy etykiet
x_dummy = pd.get_dummies(x, columns=['cp', 'restecg', 'slope','ca','thal'])
x_dummy_binary = pd.get_dummies(x_dummy, columns=['sex', 'fbs', 'exang'])

# Implementacja standardowej funkcji logistycznej jako funckji aktywacji
def activation_function(n):
    return 1 / (1 + np.exp(-n))

def function_p(x, W=None, b=None):
    return activation_function(W*x + b)

# Implementacja funkcji kosztu na podstawie entropii krzyzowej
def cost_function(x,y):
    -y*np.log(function_p(x)) - (1 - y)*np.log(1 - function_p(x))

# Backup function
def compute_cost(y, y_hat):
    m = len(y)
    cost = (-1 / m) * np.sum(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat))
    return cost

In [54]:
# Podział danych na zbiór uczący i testowy
x_train, x_test, y_train, y_test = train_test_split(x_dummy_binary, y_binary, test_size=0.3, random_state=268555)

# Normalizacja cech
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Model regresji logistycznej
model = LogisticRegression(max_iter=1000)

# Trenowanie modelu na danych uczących
model.fit(x_train, np.ravel(y_train))   # funkcja ravel zmiania macierz o wymiarach nx1 na macierz o wymiarach 1xn

# Predykcja na zbiorze testowym
y_pred = model.predict(x_test)

# Weryfikacja za pomocą metryk
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.9000
Precision: 0.9005
F1 Score: 0.8992


In [57]:
# Funkcja sigmoidalna
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Funkcja kosztu (entropia krzyżowa)
def compute_cost(y, y_hat):
    m = len(y)
    cost = (-1 / m) * np.sum(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat))
    return cost

# Pochodna funkcji kosztu (gradient dla wag i biasu)
def gradient_descent(x, y, learning_rate, iterations):
    m, n = x.shape
    W = np.zeros(n)  # Wagi
    b = 0  # Bias
    costs = []  # Do zapisywania wartości funkcji kosztu

    for i in range(iterations):
        # Predykcje modelu
        z = np.dot(x, W) + b
        y_hat = sigmoid(z)
        
        # Obliczanie gradientów
        dW = (1 / m) * np.dot(x.T, (y_hat - y))
        db = (1 / m) * np.sum(y_hat - y)
        
        # Aktualizacja wag
        W -= learning_rate * dW
        b -= learning_rate * db
        
        # Obliczanie i zapis kosztu co 100 iteracji
        if i % 100 == 0:
            cost = compute_cost(y, y_hat)
            costs.append(cost)
            print(f"Iteration {i}: Cost {cost:.4f}")
    
    return W, b, costs

# Funkcja predykcji
def predict(x, W, b):
    z = np.dot(x, W) + b
    y_hat = sigmoid(z)
    return np.where(y_hat >= 0.5, 1, 0)

# Trening modelu
learning_rate = 0.01
iterations = 1000
W, b, costs = gradient_descent(x_train, np.ravel(y_train), learning_rate, iterations)

# Predykcja na zbiorze testowym
y_pred = predict(x_test, W, b)

# Ewaluacja modelu
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")


Iteration 0: Cost 0.6931
Iteration 100: Cost 0.4403
Iteration 200: Cost 0.3895
Iteration 300: Cost 0.3696
Iteration 400: Cost 0.3589
Iteration 500: Cost 0.3521
Iteration 600: Cost 0.3473
Iteration 700: Cost 0.3437
Iteration 800: Cost 0.3409
Iteration 900: Cost 0.3386
Accuracy: 0.9000
Precision: 0.9091
F1 Score: 0.8696
